In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestRegressor

In [2]:
fireman = pd.read_excel('Fire_cleaned.xlsx')
target = pd.read_csv(r'D:\Disaster_Management\disasterManage\Temp_models\fire_archive.csv')

In [3]:
fireman

,latitude,longitude,brightness,satellite,frp,daynight,Type_2,Type_3,scan_binned,year,month,date
0,-11.8070,142.0583,313.0,1,6.6,1,0,0,1,2019,8,1
1,-11.7924,142.0850,319.3,1,11.3,1,0,0,1,2019,8,1
2,-12.8398,132.8744,311.6,1,23.1,1,0,0,4,2019,8,1
3,-14.4306,143.3035,310.1,1,6.5,1,0,0,2,2019,8,1
4,-12.4953,131.4897,310.3,1,27.6,1,0,0,4,2019,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
36006,-28.0656,131.6334,312.7,0,14.2,0,0,0,2,2019,9,30
36007,-28.2625,128.5494,304.9,0,6.4,0,0,0,1,2019,9,30
36008,-28.2941,128.5123,301.7,0,5.5,0,0,0,1,2019,9,30
36009,-28.3001,128.5530,303.6,0,5.7,0,0,0,1,2019,9,30


In [4]:
y = target['confidence']
Xtrain, Xtest, ytrain, ytest = train_test_split(fireman.iloc[:, :500], y, test_size=0.2)

In [5]:
#first the normal random forest model

In [6]:
random_model = RandomForestRegressor(n_estimators=300, random_state = 42, n_jobs = -1)
#number of tress N_estimators
#random state for consistency of output
#Cores to be used for process is n_jobs

In [7]:
#fitting the model according to the preictions
random_model.fit(Xtrain, ytrain)

y_pred = random_model.predict(Xtest)

#Checking the accuracy
random_model_accuracy = round(random_model.score(Xtrain, ytrain)*100,2)
print(round(random_model_accuracy, 2), '%')

95.32 %


In [8]:
random_model_accuracy1 = round(random_model.score(Xtest,ytest)*100,2)
print(round(random_model_accuracy1,2),'%')


65.11 %


In [9]:
import pickle 
saved_model = pickle.dump(random_model, open('ForestModelOld.pickle','wb'))

In [10]:
random_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [11]:
from sklearn.model_selection import RandomizedSearchCV

In [12]:
n_estimators = [int(x) for x in np.linspace(start = 300, stop = 500, num = 20)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(15, 35, num = 7)]
max_depth.append(None)

min_samples_split = [2, 3, 5]

min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
                }
print(random_grid)

{'n_estimators': [300, 310, 321, 331, 342, 352, 363, 373, 384, 394, 405, 415, 426, 436, 447, 457, 468, 478, 489, 500], 'max_features': ['auto', 'sqrt'], 'max_depth': [15, 18, 21, 25, 28, 31, 35, None], 'min_samples_split': [2, 3, 5], 'min_samples_leaf': [1, 2, 4]}


In [13]:
rf_random = RandomizedSearchCV(estimator = random_model, param_distributions = random_grid, 
                                n_iter = 50, cv = 3, verbose=2, random_state=42)
# Fit the random search model
rf_random.fit(Xtrain, ytrain)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END max_depth=18, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=415; total time=   4.8s
[CV] END max_depth=18, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=415; total time=   2.7s
[CV] END max_depth=18, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=415; total time=   2.6s
[CV] END max_depth=28, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=436; total time=   3.5s
[CV] END max_depth=28, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=436; total time=   3.5s
[CV] END max_depth=28, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=436; total time=   3.5s
[CV] END max_depth=35, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=478; total time=   2.3s
[CV] END max_depth=35, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=47

RandomizedSearchCV(cv=3,
                   estimator=RandomForestRegressor(n_estimators=300, n_jobs=-1,
                                                   random_state=42),
                   n_iter=50,
                   param_distributions={'max_depth': [15, 18, 21, 25, 28, 31,
                                                      35, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 3, 5],
                                        'n_estimators': [300, 310, 321, 331,
                                                         342, 352, 363, 373,
                                                         384, 394, 405, 415,
                                                         426, 436, 447, 457,
                                                         468, 478, 489, 500]},
                   random_state=42, verbose=2)

In [14]:
rf_random.best_params_

{'n_estimators': 478,
 'min_samples_split': 3,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 35}

In [15]:
random_new = RandomForestRegressor(n_estimators = 394, min_samples_split = 2, min_samples_leaf = 1, max_features = 'sqrt',
                                      max_depth = 25, bootstrap = True)

In [16]:
random_new.fit(Xtrain, ytrain)

RandomForestRegressor(max_depth=25, max_features='sqrt', n_estimators=394)

In [17]:
y_pred1 = random_new.predict(Xtest)

In [18]:
random_model_accuracy1 = round(random_new.score(Xtrain, ytrain)*100,2)
print(round(random_model_accuracy1, 2), '%')

95.35 %


In [19]:
random_model_accuracy2 = round(random_new.score(Xtest, ytest)*100,2)
print(round(random_model_accuracy2, 2), '%')

67.09 %


In [20]:
saved_model = pickle.dump(random_new, open('ForestModel.pickle','wb'))

In [22]:

reg_from_pickle = pickle.load(saved_model)

TypeError: file must have 'read' and 'readline' attributes

In [23]:
import bz2

compressionLevel = 9
source_file = 'ForestModel.pickle' # this file can be in a different format, like .csv or others...
destination_file = 'ForestModel.bz2'

with open(source_file, 'rb') as data:
    tarbz2contents = bz2.compress(data.read(), compressionLevel)
    
fh = open(destination_file, "wb")
fh.write(tarbz2contents)
fh.close()